<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [7]</a>'.</span>

In [1]:
#| hide
import sys
sys.path.append('../.venv/lib/python3.9/site-packages/')

# Model Salience Estimation

Model salience has been used extensively in the XAI community to help interpret why models make their decisions. One of the most popular salience methods is Gradient-Weighted Class Activation Mappings (Grad-CAMs), which visulaizes the gradients of the predicted class at the final convolutional layer.

In [2]:
import torch
import dvc.api

/afs/crc.nd.edu/group/TAI/Users/painswor/Explain2Me-Framework-Example/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
FILENAME: str = "ffhq_aligned/00001.png"

In [4]:
# Parameters
FILENAME = "StyleGAN-1/000700.png"


## Necessary Arguments

In [5]:
model_path = "../models/weights/Densenet/densenet_cyborg_1/Logs/final_model.pth"
device = torch.device('cpu')
network = "densenet"

## Loading the Model

In [6]:
import torchvision.models as models
import torch.nn as nn

First, load in the weights

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [7]:
weights = torch.load(model_path, map_location=device)

FileNotFoundError: [Errno 2] No such file or directory: '../models/weights/Densenet/densenet_cyborg_1/Logs/final_model.pth'

Now, load weights on a single binary DesNet121 model

In [ ]:
if network == "resnet":
    im_size = 224
    model = models.resnet50(pretrained=True)
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, 2)
elif network == "inception":
    im_size = 299
    model = models.inception_v3(pretrained=True,aux_logits=False)
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, 2)
elif network == "xception":
    im_size = 299
    model, *_ = model_selection(modelname='xception', num_out_classes=2)
else: # else DenseNet
    im_size = 224
    model = models.densenet121(pretrained=True)
    num_ftrs = model.classifier.in_features
    model.classifier = nn.Linear(num_ftrs, 2)

In [ ]:
target_layers = [model.features[-1]]
model.load_state_dict(weights['state_dict'])
model = model.to(device)
model.eval()

## Preprocessing

In [ ]:
import torchvision.transforms as transforms

In [ ]:
if network == "xception":
    # Transformation specified for the pre-processing
    transform = transforms.Compose([
                transforms.Resize([im_size, im_size]),
                transforms.ToTensor(),
                transforms.Normalize([0.5]*3, [0.5]*3)
            ])
else:
    # Transformation specified for the pre-processing
    transform = transforms.Compose([
                transforms.Resize([im_size, im_size]),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            ])

In [ ]:
imageScores = []
sigmoid = nn.Sigmoid()

## Creating GradCam

In [ ]:
from PIL import Image
from torchcam.methods import GradCAM
from torchcam.methods import GradCAMpp
import skimage.io

Laod the image

In [ ]:
gradcamType = "gradcam"
img_path = f'../data/images/{FILENAME}'

In [ ]:
# Loading in the image
image = Image.open(img_path).convert('RGB')

# Image transformation
input_tensor = transform(image)

Generate GradCAM

In [ ]:
if gradcamType == "gradcam":
    #cam_extractor = GradCAM(model, target_layers)
    cam_extractor = GradCAM(model)
elif gradcamType == "gradcamPP":
    cam_extractor = GradCAMpp(model, target_layers)

## Processing GradCAM

In [ ]:
from torchvision.transforms.functional import normalize, resize, to_pil_image
from torchcam.utils import overlay_mask

Preprocess your data and feed it to the model

In [ ]:
size = 'upscaled'

In [ ]:
out = model(input_tensor.unsqueeze(0))

Retrieve the CAM by passing class index and model output

In [ ]:
activation_map = cam_extractor(out.squeeze(0).argmax().item(), out)

In [ ]:
cam = to_pil_image(activation_map[0].squeeze(0), mode='F')
cam = overlay_mask(image, cam, alpha=0.50)
if size == 'upscaled':
    cam = cam.resize((im_size, im_size), resample = Image.BICUBIC)

## Display GradCAM

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid

In [ ]:
original_image = Image.open(img_path).convert('RGB')
overlayed_cam = cam

fig = plt.figure(figsize=(10., 8.), facecolor='white')
grid = ImageGrid(fig, 111,  # similar to subplot(111)
                 nrows_ncols=(1, 2),  # creates 2x2 grid of axes
                 axes_pad=0.1,  # pad between axes in inch.
                 aspect=1.2
                 )

for index, (ax, im) in enumerate(zip(grid, [original_image, overlayed_cam])):
    # Iterating over the grid returns the Axes.
    ax.axis('off')
    ax.grid(None)
    ax.imshow(im)
    #ax.set_title(predictions[index], fontsize=45)
fig.set_size_inches(32, 18)
plt.show()

In [ ]:
save_directory = f'../output/cams/{img_name}'
cam.save(save_directory)